In [2]:
# 📦 Install and import required packages
!pip install nltk scikit-learn torch

import pandas as pd
import nltk
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem.isri import ISRIStemmer
from torch.utils.data import TensorDataset, DataLoader

# 🔁 Download NLTK resources
nltk.download('stopwords')

# ✅ Load your dataset
df = pd.read_csv("data_semantically_scored.csv")  # ← Replace with your filename

# ✅ Step 2: Preprocessing (tokenize, remove stopwords, stem, discretize)
tokenizer = ToktokTokenizer()
stemmer = ISRIStemmer()
stop_words = set(nltk.corpus.stopwords.words('arabic'))

def preprocess(text):
    tokens = tokenizer.tokenize(str(text))
    tokens = [t for t in tokens if t.isalpha() and t not in stop_words]
    stemmed = [stemmer.stem(t) for t in tokens]
    return ' '.join(stemmed)

df['processed'] = df['title'].apply(preprocess)

# Discretize score into 5 bins for classification
# Discretize the score into 5 bins and handle NaNs safely
df['score_bin'] = pd.cut(df['score'], bins=[0, 2, 4, 6, 8, 10], labels=[0, 1, 2, 3, 4])

# Drop any rows with NaN bins
df = df.dropna(subset=['score_bin'])

# Convert to int after removing NaNs
df['score_bin'] = df['score_bin'].astype(int)

# ✅ Step 3: Vectorization
vectorizer = TfidfVectorizer(max_features=500)
X = vectorizer.fit_transform(df['processed']).toarray()
y = df['score_bin'].values

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16)

# ✅ Step 3: Model Definition
class TextClassifier(nn.Module):
    def __init__(self, model_type='rnn', input_size=500, hidden_size=64, output_size=5):
        super().__init__()
        self.rnn_type = model_type
        self.hidden = hidden_size

        if model_type == 'gru':
            self.rnn = nn.GRU(input_size, hidden_size, batch_first=True)
        elif model_type == 'lstm':
            self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        elif model_type == 'birnn':
            self.rnn = nn.RNN(input_size, hidden_size, batch_first=True, bidirectional=True)
        else:
            self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)

        self.fc = nn.Linear(hidden_size * (2 if model_type == 'birnn' else 1), output_size)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add sequence dimension
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

# ✅ Training function
def train_model(model_type):
    model = TextClassifier(model_type)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(5):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    return model

# ✅ Step 4: Evaluation
def evaluate_model(model):
    model.eval()
    all_preds, all_true = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.numpy())
            all_true.extend(labels.numpy())
    print("Accuracy:", accuracy_score(all_true, all_preds))
    print(classification_report(all_true, all_preds))

# ✅ Train & Evaluate All Models
for model_type in ['rnn', 'birnn', 'gru', 'lstm']:
    print(f"===== {model_type.upper()} Evaluation =====")
    model = train_model(model_type)
    evaluate_model(model)


===== RNN Evaluation =====


[nltk_data] Downloading package stopwords to /home/med/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Accuracy: 0.453416149068323
              precision    recall  f1-score   support

           0       0.67      0.12      0.20        34
           1       0.45      0.90      0.60        69
           2       0.41      0.13      0.20        52
           3       0.00      0.00      0.00         6

    accuracy                           0.45       161
   macro avg       0.38      0.29      0.25       161
weighted avg       0.47      0.45      0.36       161

===== BIRNN Evaluation =====


/home/med/jupyter_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/med/jupyter_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/med/jupyter_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.484472049689441
              precision    recall  f1-score   support

           0       0.50      0.38      0.43        34
           1       0.51      0.71      0.59        69
           2       0.41      0.31      0.35        52
           3       0.00      0.00      0.00         6

    accuracy                           0.48       161
   macro avg       0.36      0.35      0.34       161
weighted avg       0.46      0.48      0.46       161

===== GRU Evaluation =====


/home/med/jupyter_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/med/jupyter_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/med/jupyter_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.43478260869565216
              precision    recall  f1-score   support

           0       1.00      0.03      0.06        34
           1       0.44      0.93      0.59        69
           2       0.38      0.10      0.15        52
           3       0.00      0.00      0.00         6

    accuracy                           0.43       161
   macro avg       0.45      0.26      0.20       161
weighted avg       0.52      0.43      0.32       161

===== LSTM Evaluation =====


/home/med/jupyter_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/med/jupyter_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/med/jupyter_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.43478260869565216
              precision    recall  f1-score   support

           0       1.00      0.03      0.06        34
           1       0.43      1.00      0.60        69
           2       0.00      0.00      0.00        52
           3       0.00      0.00      0.00         6

    accuracy                           0.43       161
   macro avg       0.36      0.26      0.16       161
weighted avg       0.40      0.43      0.27       161



/home/med/jupyter_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/med/jupyter_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/med/jupyter_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
